# Tutorial for AFMpy.REC.IREC

## Imports

In [ ]:
# Standard library imports
import json
import logging
from pystackreg import StackReg

# Third party imports
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

# AFMpy imports
from AFMpy import Stack, DL, Plotting, REC, SSIM

## Configure Logging

In [ ]:
# Load the preconfigured logging settings
with open('logs/IREC_Tutorial_LoggingConfig.json', 'r') as f:
    LOGGING_CONFIG = json.load(f)

# Set up the logging configuration
logging.config.dictConfig(LOGGING_CONFIG)

## Matplotlib Configuration

Included within the ```Plotting``` module are functions for creating the high quality figures. A default configuration that matches the figures in the publication is activated by running the following function.

In [ ]:
# Configure the rcParams for the plots
Plotting.configure_formatting()

## Check if the GPU is accessible by Tensorflow

This tutorial uses Tensorflow and Keras in its deep learning algorithms. The performance, especially for large image stacks is substantially degraded when not using the GPU, so it is highly reccomended to use the GPU if available. The helper function ```DL.is_gpu_available``` will check to see if Tensorflow has GPU access. 

In [ ]:
# Check to see if the GPU is available
if DL.is_gpu_available():
    print('GPU is accessible by tensorflow.')
else:
    print('GPU is NOT accessible by tensorflow. If you want to use GPU, please check your AFMpy version and tensorflow installation.')

## Load the Stacks

Here we load the compressed pickle file of our unaligned cytoplasmic/periplasmic stacks scanned with a 2nm tip at 4Å/px (UC-20-4 and UP-20-4 respectively). A comprehensive explanation of the loading functions is available in the ```LAFM``` tutorial.

In [ ]:
# Set the filepath for the public key to verify the integrity of the stacks.
PUBLIC_KEY_FILEPATH = '../common/keys/Tutorial_Public.pub'

# Load the cytoplasmic and periplasmic stacks
cytoplasmic_stack = Stack.Stack.load_compressed_pickle(pickle_filepath = '../common/stacks/Example_UC-20-4.xz',
                                                       public_key_filepath = PUBLIC_KEY_FILEPATH)

periplasmic_stack = Stack.Stack.load_compressed_pickle(pickle_filepath = '../common/stacks/Example_UP-20-4.xz',
                                                       public_key_filepath = PUBLIC_KEY_FILEPATH)

Display the metadata of each stack.

In [ ]:
print('Cytoplasmic Stack Metadata:')
cytoplasmic_stack.display_metadata()
print('Periplasmic Stack Metadata:')
periplasmic_stack.display_metadata()

## Prepare the Convolutional Autoencoders

Prepare the convolutional autoencoder (CAE) for deep spectral clustering.

A comprehensive description of the ```ConvolutionalAutoencoder``` object is available in the ```DSC``` tutorial.

In [ ]:
# Determine the input shape for the CAE. It should be (width,height,channels). In our case (64,64,1)
cyto_input_shape = (*cytoplasmic_stack.images.shape[1:], 1)
peri_input_shape = (*periplasmic_stack.images.shape[1:], 1)

# Set the configurations for the Convolutional Autoencoder. Adjust the parameters as needed.
compile_config = DL.CompileConfig(optimizer = 'adam', loss = DL.Losses.combined_ssim_loss)
fit_config = DL.FitConfig(epochs = 25, batch_size = 32, verbose = 1)
predict_config = DL.PredictConfig(batch_size = 32, verbose = 1)

# Create the Convolutional Autoencoder models to train with our data.
cytoplasmic_CAE = DL.DefaultCAE(input_shape = cyto_input_shape, compile_config = compile_config, fit_config = fit_config, predict_config = predict_config)
periplasmic_CAE = DL.DefaultCAE(input_shape = peri_input_shape, compile_config = compile_config, fit_config = fit_config, predict_config = predict_config)

## Create the StackReg object

Because the image stacks are unaligned, they must undergo some registration before clustering. Image registration is handled by ```pystackreg```. We create an instance of the ```StackReg``` object with the registration type ```StackReg.RIGID_BODY```.

In [ ]:
sr = StackReg(StackReg.RIGID_BODY)

## Apply Iterative Registration and Clustering

### Cytoplasmic IREC

#### Train the Model and Apply IREC

The ```REC.IREC``` function applies Iterative Registration and Clustering (IREC) to a given ```Stack``` object using a specified ```ConvolutionalAutoencoder```. This method iteratively performs image registration and spectral clustering, refining clusters until convergence.

Key parameters include:

- ```cae```: The convolutional autoencoder model used to generate latent feature vectors (LFVs).

- ```sr```: The registration method (default is rigid-body registration) used to align images.

- ```reference_index```: Initial index of the reference image for registration.

- ```n_clusters```: The desired number of clusters to form.

- ```k_neighbors```: Defines the local neighborhood size for locally scaled affinity matrix calculation.

- ```max_iterations```: Limits the maximum number of iterations for the iterative registration and clustering process.

During each iteration, the stack is registered relative to optimal reference images selected by maximizing silhouette scores, and spectral clustering is subsequently applied to refine cluster assignments. Iterations continue until clusters converge or the maximum iteration limit is reached.

The function returns a list of new ```Stack``` objects, each representing a refined conformational cluster.

After we apply the IREC algorithm, we generate the mean and LAFM images for each cluster.

In [ ]:
cytoplasmic_stack.shuffle()

cyto_clusters = REC.IREC(cytoplasmic_stack,
                         cytoplasmic_CAE,
                         sr = sr,
                         n_clusters = 3,
                         k_neighbors = 7,
                         max_iterations = 5)

# Calculate the mean images for each cluster
for cluster in cyto_clusters:
    cluster.calc_mean_image() 
    cluster.calc_LAFM_image(target_resolution = (96, 96), sigma = 2.25)

#### Load and Process the Cytoplasmic Benchmark Stack

Load the benchmark stacks and register and cluster the stacks according to the optimal registration references found by applying IREC to the AC-20-4 stack. Each benchmark cluster has its LAFM image generated..

In [ ]:
benchmark_cytoplasmic_stack = Stack.Stack.load_compressed_pickle(pickle_filepath = '../common/stacks/Example_AC-2-2.xz',
                                                                 public_key_filepath = PUBLIC_KEY_FILEPATH)

benchmark_cyto_clusters = []
for cluster in cyto_clusters:
    ref = cluster.indexes == cluster.get_metadata('registration_reference')
    benchmark_images = benchmark_cytoplasmic_stack.images[cluster.indexes]
    benchmark_ref = benchmark_images[ref][0]
    registered_bechmark_images = REC.register_image_stack(benchmark_ref, benchmark_images, sr = sr)
    bechmark_cluster = Stack.Stack(images = registered_bechmark_images, resolution = benchmark_cytoplasmic_stack.resolution, indexes = cluster.indexes)
    bechmark_cluster.calc_LAFM_image(target_resolution = (192,192), sigma = 2.25)
    benchmark_cyto_clusters.append(bechmark_cluster)

#### Display the Cytoplasmic Clustered LAFM Images

Plot the mean, LAFM, and benchmark LAFM images from each cluster.

In [ ]:
# Set the color range based upon the maximum LAFM image value across all clusters.
vmin = 0
vmax = np.max([cluster.LAFM_image for cluster in cyto_clusters])

# Create the figure.
fig, ax = plt.subplots(len(cyto_clusters), 3, figsize = (9, 3 * len(cyto_clusters)))

# Turn off the tick marks
for axis in ax.ravel():
    axis.set_xticks([])
    axis.set_yticks([])

# Set the axis labels 
ax[0,0].set_title('Mean Image', fontsize = 16)
ax[0,1].set_title('LAFM Image', fontsize = 16)
ax[0,2].set_title('Benchmark Image', fontsize = 16)

# Plot the mean images, LAFM images, and benchmark images for each cluster.
for i in range(len(cyto_clusters)):
    # Set the title for the cluster
    ax[i,0].set_ylabel(f'Cluster {i}', fontsize = 16)

    # Plot the mean image and its scalebar
    cyto_clusters[i].plot_mean_image(ax = ax[i,0], cmap = Plotting.LAFMcmap, vmin = vmin, vmax = vmax)
    Plotting.add_scalebar(10/cytoplasmic_stack.resolution, label = '1nm', ax = ax[i,0])

    # Plot the LAFM image and its scalebar
    cyto_clusters[i].plot_LAFM_image(ax = ax[i,1], cmap = Plotting.LAFMcmap, vmin = vmin, vmax = vmax)
    Plotting.add_scalebar(30/cytoplasmic_stack.resolution, label = '1nm', size_vertical = 3/8, ax = ax[i,1])

    # Plot the benchmark image and its scalebar
    benchmark_cyto_clusters[i].plot_LAFM_image(ax = ax[i,2], cmap = Plotting.LAFMcmap, vmin = vmin, vmax = vmax)
    Plotting.add_scalebar(60/cytoplasmic_stack.resolution, label = '1nm', size_vertical = 6/8, ax = ax[i,2])

# Add the colorbars to the right of the images.
cbar_ax = fig.add_axes([0.91, 0.11, 0.03, 0.77])
Plotting.draw_colorbar_to_ax(vmin, vmax, Plotting.LAFMcmap,
                             label = 'Height (Å)', cbar_ax = cbar_ax)

plt.show()

#### Calculate the SSIM Between the LAFM Images

Finally, we evaluate the increase in image quality via LAFM by calculating the Structural Similarity Index Measure (SSIM) between the mean/LAFM image and the benchmark LAFM. 

In [ ]:
ssim_df = {
    'Mean Image':[SSIM.registered_SSIM(cv2.resize(cyto_clusters[index].mean_image, (192,192)), benchmark_cyto_clusters[index].LAFM_image, sr = sr) for index in range(len(cyto_clusters))],
    'LAFM Image':[SSIM.registered_SSIM(cv2.resize(cyto_clusters[index].LAFM_image, (192,192)), benchmark_cyto_clusters[index].LAFM_image, sr = sr) for index in range(len(cyto_clusters))]
}
ssim_df = pd.DataFrame(ssim_df)
ssim_df.index = [f'Cluster {index}' for index in range(len(cyto_clusters))]
display(ssim_df.round(2))

### Periplasmic IREC

Repeat the IREC algorithm for the periplasmic stack.

In [ ]:
periplasmic_stack.shuffle()

peri_clusters = REC.IREC(periplasmic_stack,
                         periplasmic_CAE,
                         sr = sr,
                         n_clusters = 2,
                         k_neighbors = 7,
                         max_iterations = 5)

# Calculate the mean images for each cluster
for cluster in peri_clusters:
    cluster.calc_mean_image() 
    cluster.calc_LAFM_image(target_resolution = (96, 96), sigma = 2.25)

#### Load and Process the Periplasmic Benchmark Stack

In [ ]:
benchmark_periplasmic_stack = Stack.Stack.load_compressed_pickle(pickle_filepath = '../common/stacks/Example_AP-2-2.xz',
                                                                 public_key_filepath = PUBLIC_KEY_FILEPATH)

benchmark_peri_clusters = []
for cluster in peri_clusters:
    ref = cluster.indexes == cluster.get_metadata('registration_reference')
    benchmark_images = benchmark_periplasmic_stack.images[cluster.indexes]
    benchmark_ref = benchmark_images[ref][0]
    registered_bechmark_images = REC.register_image_stack(benchmark_ref, benchmark_images, sr = sr)
    bechmark_cluster = Stack.Stack(images = registered_bechmark_images, resolution = benchmark_periplasmic_stack.resolution, indexes = cluster.indexes)
    bechmark_cluster.calc_LAFM_image(target_resolution = (192,192), sigma = 2.25)
    benchmark_peri_clusters.append(bechmark_cluster)

#### Display the Periplasmic Clustered LAFM Images

In [ ]:
# Set the color range based on the maximum LAFM image value across all periplasmic clusters
vmin = 0
vmax = np.max([cluster.LAFM_image for cluster in peri_clusters])

# Create the figure
fig, ax = plt.subplots(len(peri_clusters), 3, figsize=(9, 3 *len(peri_clusters)))

# Turn off tick marks
for axis in ax.ravel():
    axis.set_xticks([])
    axis.set_yticks([])

# Set axis labels
ax[0, 0].set_title('Mean Image',      fontsize=16)
ax[0, 1].set_title('LAFM Image',      fontsize=16)
ax[0, 2].set_title('Benchmark Image', fontsize=16)

for i in range(len(peri_clusters)):
    # Set the title for the cluster
    ax[i,0].set_ylabel(f'Cluster {i}', fontsize = 16)

    # Plot the mean image and its scalebar
    peri_clusters[i].plot_mean_image(ax = ax[i,0], cmap = Plotting.LAFMcmap, vmin = vmin, vmax = vmax)
    Plotting.add_scalebar(10/periplasmic_stack.resolution, label = '1nm', ax = ax[i,0])

    # Plot the LAFM image and its scalebar
    peri_clusters[i].plot_LAFM_image(ax = ax[i,1], cmap = Plotting.LAFMcmap, vmin = vmin, vmax = vmax)
    Plotting.add_scalebar(30/periplasmic_stack.resolution, label = '1nm', size_vertical = 3/8, ax = ax[i,1])

    # Plot the benchmark image and its scalebar
    benchmark_peri_clusters[i].plot_LAFM_image(ax = ax[i,2], cmap = Plotting.LAFMcmap, vmin = vmin, vmax = vmax)
    Plotting.add_scalebar(60/periplasmic_stack.resolution, label = '1nm', size_vertical = 6/8, ax = ax[i,2])

# Add the colorbar
cbar_ax = fig.add_axes([0.91, 0.11, 0.03, 0.77])
Plotting.draw_colorbar_to_ax(vmin, vmax, Plotting.LAFMcmap,
                             label='Height (Å)', cbar_ax=cbar_ax)

plt.show()

#### Calculate the SSIM between the LAFM Images

In [ ]:
ssim_df = {
    'Mean Image':[SSIM.registered_SSIM(cv2.resize(peri_clusters[index].mean_image, (192,192), cv2.INTER_CUBIC), benchmark_peri_clusters[index].LAFM_image, sr = sr) for index in range(len(peri_clusters))],
    'LAFM Image':[SSIM.registered_SSIM(cv2.resize(peri_clusters[index].LAFM_image, (192,192), cv2.INTER_CUBIC), benchmark_peri_clusters[index].LAFM_image, sr = sr) for index in range(len(peri_clusters))]
}
ssim_df = pd.DataFrame(ssim_df)
ssim_df.index = [f'Cluster {index}' for index in range(len(peri_clusters))]

display(ssim_df.round(2))